### Example 9 - Scrapmetal Arbitrage

In this example, we'll review the results of looking for scrapmetal arbitrage opportunites in a day of order book
data.  The parameters for our opportunity finder are the same as for the ore and ice case:

* we're looking in The Forge region; and
* we're assuming we're buying and selling from Jita IV - Moon 4 - Caldari Navy Assembly Plant.

As mentioned in the text, running the opportunity finder in a Jupyter notebook would take too long to make this
a useful example.  Instead, we've generated opportunities offline using a script \(see the text for more
details\).  We'll load the results which have the same format as in the previous example.  We'll then clean
the results and analyze the available opportunities.

Arbitrage opportunities for scrapmetal processing use the same equation as before:

$\sum_i\left(v_i \times e \times \left[ (1 - t) \times p_b(c_i) - s_t \times p_r(c_i)\right] \right) -  r_m \times p_a(r)$

where:

* $r$ - type ID of an ore or ice type to be refined.
* $r_m$ - the amount of material required for one refinement cycle.
* $e$ - refinement efficiency computed from station efficiency and player skills.
* $m(r) \rightarrow {(c_1, v_1), ..., (c_n, v_n)}$ - a map from source material to a set of refined materials $(c_i, v_i)$ where $c_i$ represents the refined material type, and $v_i$ represents the ideal quantity produced when refining $r_m$ units of $r$.
* $p_r(t)$ - the reference price for an asset \(such a price exists for all marketable EVE types\).
* $p_a(t)$ - the current best ask price for an asset \(i.e. the minimum price at which the asset can be purchased\).
* $p_b(t)$ - the current best bid price for an asset \(i.e. the maximum price at which the asset can be sold\).
* $t$ - sales tax for the refining player.
* $s_t$ - station refining tax.

The only difference is that efficiency is modified by only one skill and is therefore significantly lower than the efficiency which can be obtained when reprocessing ore or ice.  This means prices have to move more to make arbitrage ppossible.  As in our ore and ice examples, we're replacing "adjusted_price" with $p_b$.  A production quality version of this example would use `adjusted_price` instead.

In [15]:
# Standard imports
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
# EveKit imports
from evekit.reference import Client
from evekit.util import convert_raw_time

In [16]:
# We've collected results from our backtest into a CSV file with format:
#
# snapshot time, profit, gross, cost, type
#
# We'll start by reading this data into an array.
#
results_file = "single_day_scrap.csv"

opportunities = []
fin = open(results_file, 'r')
for line in fin:
    columns = line.strip().split(",")
    opportunities.append(dict(time=datetime.datetime.strptime(columns[0], "%Y-%m-%d %H:%M:%S"),
                             profit=float(columns[1]),
                             gross=float(columns[2]),
                             cost=float(columns[3]),
                             type=columns[4]))

In [19]:
# Our offline script exploits parallelism to find opportunities more quickly (but a day
# of data still takes four hours to analyze).  A side effect is that our list of opportunities
# is unsorted, so we'll first sort by opportunity time.
opportunities = sorted(opportunities, key=lambda x: x['time'])

In [20]:
# Now we're ready to clean the list, collapsing adjacent opportunities into
# their first occurrence.  We'll use the same function as before:
def clean_opportunities(opps):
    new_opps = []
    stamp_map = {}
    types = set([x['type'] for x in opps])
    # Flatten opportunites for each type
    for next_type in types:
        stamp_list = []
        last = None
        for i in [x['time'] for x in opps if x['type'] == next_type]:
            if last is None:
                # First opportunity
                stamp_list.append(i)
            elif i - last > datetime.timedelta(minutes=5):
                # Start of new run
                stamp_list.append(i)
            last = i
        stamp_map[next_type] = stamp_list
    # Rebuild opportunities by only selecting opportunities in
    # the flattened lists.
    for opp in opps:
        type = opp['type']
        if opp['time'] in stamp_map[type]:
            new_opps.append(opp)
    # Return the new opportunity list
    return new_opps

opportunities = clean_opportunities(opportunities)

In [26]:
# Let's take a look at what we have
def display_opportunities(opps):
    for opp in opps:
        profit = "{:15,.2f}".format(opp['profit'])
        margin = "{:8.2f}".format(opp['profit'] / opp['cost'] * 100)
        print("ArbOpp time=%s  profit=%s  return=%s%%  type=%s" % (str(opp['time']), profit, margin, opp['type']))
    print("Total opportunities: %d" % len(opps))

display_opportunities(opportunities)   

ArbOpp time=2017-01-10 00:00:00  profit=     273,055.01  return=    3.84%  type=Iridium Charge L
ArbOpp time=2017-01-10 00:00:00  profit=       6,315.99  return=   88.41%  type=Radio M
ArbOpp time=2017-01-10 00:00:00  profit=     447,356.54  return=   16.99%  type=Microwave M
ArbOpp time=2017-01-10 00:00:00  profit=       1,229.00  return=    0.45%  type=Small Remote Capacitor Transmitter I
ArbOpp time=2017-01-10 00:00:00  profit=      20,151.63  return=   19.18%  type=Radar ECM I
ArbOpp time=2017-01-10 00:00:00  profit=     287,096.21  return=   33.94%  type=Explosive Deflection Field I
ArbOpp time=2017-01-10 00:00:00  profit=     645,303.98  return=   15.74%  type=Large Armor Repairer I
ArbOpp time=2017-01-10 00:00:00  profit=     286,643.16  return=    8.58%  type=Heavy Capacitor Booster I
ArbOpp time=2017-01-10 00:00:00  profit=      11,783.47  return=    1.84%  type=Medium I-a Polarized Armor Regenerator
ArbOpp time=2017-01-10 00:00:00  profit=     734,193.63  return=   13.18%  ty

Despite much lower efficiency, there are still quite a few arbitrage opportunities available.  Typical scrapmetal processing efficiency will be 0.55 in an NPC station with max skills, versus 0.69575 for ore and ice reprocessing \(also with max skills\).  However, a combination of the shear number of types which can be processed together with natural market behavior leads to hundreds of opportunities in our sample day. 

In [28]:
# Now let's generate a summary to see what the total opportunity looks like.
total_profit = np.sum([x['profit'] for x in opportunities])
total_cost = np.sum([x['cost'] for x in opportunities])
total_return = total_profit / total_cost
print("Total opportunity profit: %s ISK" % "{:,.2f}".format(total_profit))
print("Total opportunity retrun: %s%% ISK" % "{:,.2f}".format(total_return * 100))

Total opportunity profit: 152,437,558.36 ISK
Total opportunity retrun: 2.78% ISK


Returns are in the same range as in the ore and ice case, but total profit is substantially higher \(about three times our result for ore and ice\).  If these results are typical, we'd expect a run rate of about 4.5B ISK/month  which is a great result given the amount of effort and risk imposed by this strategy.  The only way to improve our confidence is to run a longer backtest.  We'll take that analysis up in the next example.